In [1]:
import os; os.chdir('..')
import numpy as np
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

In [2]:
from prev_ob_models.Birgiolas2020.isolated_cells import TC2 as MC

In [3]:
mc = MC()

In [4]:
from neuron import h

vec = h.Vector()
vec.record(mc.cell.soma(0.5)._ref_v, 0.5)


h.tstop = 10000

#h.newPlotV()

In [5]:
ic = h.IClamp(mc.cell.soma(0.5))
ic2 = h.IClamp(mc.cell.soma(0.5))
ic3 = h.IClamp(mc.cell.soma(0.5))

In [6]:
ic.delay = 1000
ic.dur = 1000
ic.amp = -0.111


ic2.delay = 4000
ic2.dur = 1
ic2.amp = 1


ic3.delay = 7000
ic3.dur = 2000
ic3.amp = 0.111

In [7]:
from neuron import gui

In [8]:
h.newPlotV()
h.run()
raise

TypeError: exceptions must be old-style classes or derived from BaseException, not NoneType

	-68.307689 


In [ ]:
params = mc.params

In [ ]:
best = mc.param_values

for i, p in enumerate(params):
    params[i]["start"] = best[i]

In [ ]:
kwargs = {}
for pi, pv in enumerate(params):
    #print(pv["attr"])
    kwargs[pv["attr"]+"|"+str(pv["lists"])] = FloatSlider(
        value=pv["start"],
        min=pv["low"],
        max=pv["high"],
        step=(pv["high"]-pv["low"])/100.0,
        continuous_update=False,
        readout_format='.9f',
        description=(pv["attr"]+"|"+str(pv["lists"][0])).replace("gbar_","g")
    )
#kwargs

In [ ]:
def evaluate(**kwargs):   
    param_keys = kwargs.keys()
    
    for key in param_keys:
        attr = key.split("|")[0]
        lists = eval(key.split("|")[1])
        
        for param_list in lists:
            for sec in getattr(mc.cell, param_list):
                if attr == "tau_CaPool":
                    setattr(h, attr, kwargs[key])
                elif attr == "diam":
                    setattr(sec, attr, kwargs[key]*getattr(sec, attr))
                else:
                    setattr(sec, attr, kwargs[key])
        
    h.run()
    
    from matplotlib import pyplot as plt
    plt.plot(vec.as_numpy())
    plt.show()
    
    return mc.cell.soma(0.5).v

In [ ]:
interact(evaluate, **kwargs);